Математические функции в SQL:
1. CEILING(x) - возвращает наименьшее целое число, большее или равное x
(округляет до целого числа в большую сторону)
CEILING(4.2)=5
CEILING(-5.8)=-5
2. ROUND(x, k) - округляет значение x до k знаков после запятой, если k не указано – x округляется до целого
ROUND(4.361)=4
ROUND(5.86592,1)=5.9
3. FLOOR(x) - возвращает наибольшее целое число, меньшее или равное x
(округляет до  целого числа в меньшую сторону)
FLOOR(4.2)=4
FLOOR(-5.8)=-6
4. POWER(x, y) - возведение x в степень y
POWER(3,4)=81.0
5. SQRT(x) - квадратный корень из x
SQRT(4)=2.0
SQRT(2)=1.41...
6. DEGREES(x) - конвертирует значение x из радиан в градусы
7. RADIANS(x) - конвертирует значение x из градусов в радианы
8. ABS(x) - модуль числа х
9. PI()

Задание: Вывести название книг, состоящих ровно из 5 букв.
SELECT title FROM book 
WHERE title LIKE "_____"

Или если надо длиннее 5 символов - SELECT title FROM book 
WHERE title LIKE "______%";

Вывести названия книг, которые состоят ровно из одного слова, если считать, что слова в названии отделяются друг от друга пробелами .
SELECT title FROM book 
WHERE title NOT LIKE "% %";    

Задание: Вывести название и автора тех книг, название которых состоит из двух и более слов, а инициалы автора содержат букву «С». Считать, что в названии слова отделяются друг от друга пробелами и не содержат знаков препинания, между фамилией автора и инициалами обязателен пробел, инициалы записываются без пробела в формате: буква, точка, буква, точка. Информацию отсортировать по названию книги в алфавитном порядке.


SELECT title, author FROM book

WHERE title like '%_ %' and author like '%С.%'

ORDER BY title;

Задание: Магазин счёл, что классика уже не пользуется популярностью, поэтому необходимо в выборке:
1. Сменить всех авторов на "Донцова Дарья".
2. К названию каждой книги в начале дописать "Евлампия Романова и".
3. Цену поднять на 42%.
4. Отсортировать по убыванию цены и убыванию названия.

SELECT 'Донцова Дарья' AS author, CONCAT('Евлампия Романова и ', title) AS title, price * 1.42 AS price FROM book

ORDER BY price DESC, title DESC;

# Вложенные запросы

WHERE | HAVING выражение оператор_сравнения (вложенный запрос);

WHERE | HAVING выражение, включающее вложенный запрос;

WHERE | HAVING выражение [NOT] IN (вложенный запрос);

WHERE | HAVING выражение  оператор_сравнения  ANY | ALL (вложенный запрос).

Пример вложенного запроса: Вывести информацию о самых дешевых книгах, хранящихся на складе.

SELECT title, author, price, amount
FROM book

WHERE price = (
         SELECT MIN(price) 
         FROM book
      );


Пример: Вывести информацию (автора, название и цену) о тех книгах, цены которых превышают минимальную цену книги на складе не более чем на 150 рублей в отсортированном по возрастанию цены виде.

SELECT author, title, price FROM book

WHERE price - (SELECT MIN(price) FROM book) <=150

ORDER BY price;

Пример: Вывести информацию (автора, книгу и количество) о тех книгах, количество экземпляров которых в таблице book не дублируется.

SELECT author, title, amount FROM book

WHERE amount IN (SELECT amount FROM book
                
                GROUP BY amount
                
                HAVING COUNT(amount) = 1);

Оператор ANY (10 и 12 для примера):

amount > ANY (10, 12) эквивалентно amount > 10

amount < ANY (10, 12) эквивалентно amount < 12

amount = ANY (10, 12) эквивалентно (amount = 10) OR (amount = 12), а также amount IN  (10,12)

amount <> ANY (10, 12) вернет все записи с любым значением amount, включая 10 и 12

Оператор ALL (10 и 12 для примера):

amount > ALL (10, 12) эквивалентно amount > 12

amount < ALL (10, 12) эквивалентно amount < 10

amount = ALL (10, 12) не вернет ни одной записи, так как эквивалентно (amount = 10) AND (amount = 12)
amount <> ALL (10, 12) вернет все записи кроме тех,  в которыхamount равно 10 или 12

ОПЕРАТОРЫ МОЖНО ИСПОЛЬЗОВАТЬ ТОЛЬКО С ВЛОЖЕННЫМИ ЗАПРОСАМИ!

Пример 1: Вывести информацию о тех книгах, количество которых меньше самого маленького среднего количества книг каждого автора.

SELECT title, author, amount, price
FROM book

WHERE amount < ALL (
        
        SELECT AVG(amount) 
        
        FROM book 
        
        GROUP BY author 
      );

Пример 2: Вывести информацию о книгах(автор, название, цена), цена которых меньше самой большой из минимальных цен, вычисленных для каждого автора.

SELECT author, title, price FROM book

WHERE price < ANY (
                
        SELECT MIN(price)
                  
        FROM book

        GROUP BY author);

Пример: Посчитать сколько и каких экземпляров книг нужно заказать поставщикам, чтобы на складе стало одинаковое количество экземпляров каждой книги, равное значению самого большего количества экземпляров одной книги на складе. Вывести название книги, ее автора, текущее количество экземпляров на складе и количество заказываемых экземпляров книг. Последнему столбцу присвоить имя Заказ. В результат не включать книги, которые заказывать не нужно.

SELECT title, author, amount, ((SELECT MAX(amount) FROM book) - amount) AS Заказ FROM book

WHERE amount not in (SELECT MAX(amount) FROM book);

!!Обрати внимание, что во вложенном запросе нельяз отнять значение из основного запроса (SELECT (max(amount)-amount) FROM book) - это НЕВЕРНО

А в условии WHERE мы указываем что последнюю строчку, где кол-во экземпляров = максимальному, мы не вносим в результирующую таблицу.

Задача: Создать таблицу заказ (ordering), куда включить авторов и названия тех книг, количество экземпляров которых в таблице book меньше среднего количества экземпляров книг в таблице book. В таблицу включить столбец   amount, в котором для всех книг указать одинаковое значение - среднее количество экземпляров книг в таблице book.

CREATE TABLE ordering AS

SELECT author, title, (SELECT ROUND(AVG(amount)) FROM book) AS amount FROM book

WHERE amount < SELECT ROUND(AVG(amount)) FROM book;


Задача: Занести в таблицу fine суммы штрафов, которые должен оплатить водитель, в соответствии с данными из таблицы traffic_violation. При этом суммы заносить только в пустые поля столбца  sum_fine.

UPDATE fine f, traffic_violation tv

SET f.sum_fine = tv.sum_fine

WHERE f.violation = tv.violation and f.sum_fine is null;

Чтобы выбрать сколько-то строк случайным образом: 

ORDER BY RAND()

Чтобы выделить крайние левые n символов из строки используется функция LEFT(строка, n):

LEFT("abcde", 3) -> "abc"

Соединение строк осуществляется с помощью функции CONCAT(строка_1, строка_2):

CONCAT("ab","cd") -> "abcd"